# SQLite GUI 

# Setup

import all packages needed

In [1]:
import numpy as np
import sys
import btk
import sqlite3
import json
from tkinter import *
from tkinter import filedialog 
import os.path 

unless stated the filenames are all none values (null) and the data is not a full clean cycle.
The list numTrialslist will be populated by "Trial 1", "Trial 2", etc depending on what are uploaded

In [2]:
filename1 = None
filename2 = None
filename3 = None
filename4 = None
filename5 = None
filename6 = None
cleanleft1 = "No"
cleanright1 = "No"
cleanleft2 = "No"
cleanright2 = "No"
cleanleft3 = "No"
cleanright3 = "No"
cleanleft4 = "No"
cleanright4 = "No"
cleanleft5 = "No"
cleanright5 = "No"
cleanleft6 = "No"
cleanright6 = "No"
numTrialslist = []

Initiate the connection to the SQLite database. USe memory for testing but connect to the database when using.

In [3]:
"""Start SQLite Connection"""
conn = sqlite3.connect(':memory:')
#conn = sqlite3.connect('MotionAnalysis14.db')
c=conn.cursor()

### *TEMPORARY*
Used to create the tables. This will be in another file for actual use

**Table 1: Patient Data:** 
- ID
- Dominant_Limb
- Patient_Type
- Month
- Affected_Limb
- Filename_Gait_Trial_1
- Filename_Gait_Trial_2
- Filename_Gait_Trial_3
- Filename_Gait_Trial_4
- Filename_Gait_Trial_5
- Filename_Gait_Trial_6

**Table 2: Gait_Table:** 
- Filename
- Complete_Left_Cycle
- Complete_Right_Cycle
- Left_Footstrike_Index
- Left_FootOff_Index
- Right_Footstrike_Index
- Right_FootOff_Index

each variable has a seperate column for the limb and axis for example:
- Left_Knee_Angle_x
- Left_Knee_Angle_y
- Left_Knee_Angle_z


- {L/R}_Knee_Moment_{axis}
- {LR}_Knee_Force_{axis}
- {LR}_Knee_Power_{axis}
- {LR}_Hip_Angle_{axis}
- {LR}_Hip_Moment_{axis}
- {LR}_Hip_Force_{axis}
- {LR}_Hip_Power_{axis}
- {LR}_Ankle_Angle_{axis}
- {LR}_Ankle_Moment_{axis}
- {LR}_Ankle_Force_{axis}
- {LR}_Ankle_Power_{axis}
- {LR}_Foot_Progression_Angle_{axis}
- {LR}_GRF_{axis}
- {LR}_NGRF_{axis}
- {LR}_pelvis_Angle_{axis}
- {LR}_COM_{axis}

In [ ]:

def create_patient_data_table():
    c.execute("""CREATE TABLE IF NOT EXISTS Patient_Data
              (ID TEXT PRIMARY KEY, Dominant_Limb TEXT, Patient_Type TEXT, 
              Month TEXT, Affected_Limb TEXT,
              Filename_Gait_Trial_1 TEXT, Filename_Gait_Trial_2 TEXT, Filename_Gait_Trial_3 TEXT,
              Filename_Gait_Trial_4 TEXT,Filename_Gait_Trial_5 TEXT, Filename_Gait_Trial_6 TEXT
              )""")
              #add Filename_STS_Trial_1,.... for different movements             
def create_Gait_Table():
        c.execute("""CREATE TABLE IF NOT EXISTS Gait_Table
              (Filename TEXT, Complete_Left_Cycle TEXT, Complete_Right_Cycle TEXT,
              Left_Footstrike_Index TEXT, Left_FootOff_Index TEXT,
              Right_Footstrike_Index TEXT, Right_FootOff_Index TEXT)""")            
def columns_Gait_Table():
     for LR in ["Left", "Right"]:
         for axis in ["x", "y", "z"]:
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_Knee_Angle_{axis} TEXT""")
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_Knee_Moment_{axis} TEXT""")
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_Knee_Force_{axis} TEXT""")
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_Knee_Power_{axis} TEXT""")
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_Hip_Angle_{axis} TEXT""")
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_Hip_Moment_{axis} TEXT""")
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_Hip_Force_{axis} TEXT""")
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_Hip_Power_{axis} TEXT""")
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_Ankle_Angle_{axis} TEXT""")
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_Ankle_Moment_{axis} TEXT""")
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_Ankle_Force_{axis} TEXT""")
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_Ankle_Power_{axis} TEXT""")
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_Foot_Progression_Angle_{axis} TEXT""")
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_GRF_{axis} TEXT""")
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_NGRF_{axis} TEXT""")
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_pelvis_Angle_{axis} TEXT""")
             c.execute(f"""ALTER TABLE Gait_Table ADD COLUMN {LR}_COM_{axis} TEXT""")             
              
create_patient_data_table()
create_Gait_Table()
columns_Gait_Table()

Setup the GUI window (root)\
Root > MainFrame > Canvas > ScrollFrame > all the internal frames \
The scrollbar is in the MainFrame but Scrolls the canvas (and therefore the ScrollFrame and all the frames within)


In [4]:
"""Setup GUI and Frames"""
root = Tk()
MainFrame = Frame(root, bg = "blue")
MainFrame.pack(fill = BOTH) 

mycanvas = Canvas(MainFrame)
mycanvas.pack(side = LEFT, fill = BOTH, expand =True)
scroll_y = Scrollbar(MainFrame, orient = "vertical", command = mycanvas.yview)
scroll_y.pack(side = RIGHT, fill = BOTH)
mycanvas.configure(width = 860, height = 500)
mycanvas.configure(yscrollcommand = scroll_y.set)
mycanvas.bind('<Configure>', lambda e: mycanvas.configure(scrollregion = mycanvas.bbox("all")) )
ScrollFrame = Frame(mycanvas, bg = "DodgerBlue4")
ScrollFrame.pack(fill = BOTH)
mycanvas.create_window((0,0), window = ScrollFrame, anchor = "nw")

FrameLabel = Frame(ScrollFrame, bg = "DodgerBlue3", bd = 6)
FrameLabel.pack()
FrameTrial1 = Frame(ScrollFrame, bg = "ivory3", bd = 6)
FrameTrial1.pack()
FrameTrial2 = Frame(ScrollFrame, bg = "ghost white", bd = 6)
FrameTrial2.pack()
FrameTrial3 = Frame(ScrollFrame, bg = "ivory3", bd = 6)
FrameTrial3.pack()
FrameTrial4 = Frame(ScrollFrame, bg = "ghost white", bd = 6)
FrameTrial4.pack()
FrameTrial5 = Frame(ScrollFrame, bg = "ivory3", bd = 6)
FrameTrial5.pack()
FrameTrial6 = Frame(ScrollFrame, bg = "ghost white", bd = 6)
FrameTrial6.pack()
FrameInfo = Frame(ScrollFrame, bg = "DodgerBlue3", bd = 6)
FrameInfo.pack()
FrameClose = Frame(ScrollFrame, bg = "DeepSkyBlue2")
FrameClose.pack()

# Define Functions

Repeated 6 times

- Updates the previously defined filename from "None" to the full file directory
- updates the label to show "File Opened: (filename)"
- Ads to the list NumTrialsList

### *Needs work*
- remove the directory to just have the filename
- change directory to hispital directory



In [ ]:
def browseFiles1(): 
    global filename1
    filename1 = os.path.basename(filedialog.askopenfilename(
        initialdir = "C:/Users/johan/Documents/MEng FYP/C3D_files", 
        title = "Select a File", 
        filetypes = (("C3D files","*.c3d*"),
                     ("all files", "*.*"))))
    label_file_explorer1.configure(text="File Opened: "+filename1)
    numTrialslist.append("Trial 1")

**extractintoDict**
1. Takes in the filename and the TrialNum (ie. "Trial 1"). 
2. Creates a btk acquisition object (trialacq)
3. Creates two more layers of the dictionary (Trialnum and Limb) and calls the function extract values to populate these with another layer of dictionary and the arrays corresponding to the variables
4. Creates dictionary keys with empty list values within Trialnum for the index vaues of the events (footstrike and foot off)
5. Gets the "start frame" number which can be used to connect the index of the events to the index of the values collected


6. For every event recorded, the loop identifies whether its a Foot Strike of Foot off using GetLabel() and an if statement. Within this loop the GetContext() and another if statements differentiates whether it is a left or right limb event. The Frame value of each event is found using GetFrame. The start frame value found in step 5 is subtracted to from the event frame number to find the index that the event occured at. This value is stored in the dictionary in the initialised lists we set up in step 4. More than one event index can be stored per eventtype/limb


5. An if statement determines if there is at least 2 of the same limb foot strike value in the list. If yes, there is a complete footstrike to footstrike cycle and therefore the data can be normalised using the cutcycle function to exclude any data outside the full cycle. If not a full cycle, nothing is cut from the data.

6. For loop that sorts th events in chronological order

7. The final for loop breaks the arrays stored in the dictionary into seperate x, y, and z arrays and turns them into lists


8. Gets the Paitent ID from the file and updates the entry widget in the patient info section. This technically overwrites itself each time but should all be the same


**extractvalues**
simply extracts all the data from each variable and populates the dictionary

**cutcycle**
fucntion takes the index of the first and second footstrikes and cuts the data to include only the values between them

In [ ]:

#EXCTRACT AND NORMALISE DATA
def cutcycle(footstrike, data):
    footstrike1 = footstrike[0]
    footstrike2 = footstrike[1]
    data1 = (data[footstrike1:footstrike2])
    return data1
    
def extractvalues(trial):
    #extracts al the arrays
    #can be used for all trials, just needs to be instantiated with the correct trial
    x = dict();  
    x["KneeAngle"]= (trial.GetPoint("LKneeAngles")).GetValues()
    x["KneeForce"]= (trial.GetPoint("LKneeForce")).GetValues()
    x["KneeMoment"]= (trial.GetPoint("LKneeMoment")).GetValues()
    x["KneePower"]= (trial.GetPoint("LKneePower")).GetValues() 
    x["HipAngle"]= ((trial.GetPoint("LHipAngles")).GetValues())
    x["HipForce"]= (trial.GetPoint("LHipForce")).GetValues()
    x["HipMoment"]= (trial.GetPoint("LHipMoment")).GetValues()
    x["HipPower"]= (trial.GetPoint("LHipPower")).GetValues() 
    x["AnkleAngle"]= (trial.GetPoint("LAnkleAngles")).GetValues()
    x["AnkleForce"]= (trial.GetPoint("LAnkleForce")).GetValues()
    x["AnkleMoment"]= (trial.GetPoint("LAnkleMoment")).GetValues()
    x["AnklePower"]= (trial.GetPoint("LAnklePower")).GetValues()    
    x["FootProgressAngle"] = (trial.GetPoint("LFootProgressAngles")).GetValues()
    x["GRF"] = (trial.GetPoint("LGroundReactionForce")).GetValues()
    x["NGRF"] = (trial.GetPoint("LNormalisedGRF")).GetValues()
    x["pelvisAngle"] = (trial.GetPoint("LPelvisAngles")).GetValues() 
    x["COM"] = (trial.GetPoint("CentreOfMass")).GetValues()
    return x

def extractintoDict(filename, TrialNum):
    reader = btk.btkAcquisitionFileReader()
    reader.SetFilename(filename) # set a filename to the reader
    reader.Update()
    trialacq = reader.GetOutput() # is the btk aquisition object
    patient["Gait"][TrialNum] ={}
    patient["Gait"][TrialNum].update({"Left": extractvalues(trialacq)})
    patient["Gait"][TrialNum].update({"Right": extractvalues(trialacq)})
    patient["Gait"][TrialNum].update({"lfootstrikeFrame":[]})
    patient["Gait"][TrialNum].update({"rfootstrikeFrame":[]})
    patient["Gait"][TrialNum].update({"lfootoffFrame":[]})
    patient["Gait"][TrialNum].update({"rfootoffFrame":[]})
    patient["Gait"][TrialNum].update({"startframe":trialacq.GetFirstFrame()})#first frame of data
    # #instantiate the nested dict to input in the footstrike etc

    
    for i in range(trialacq.GetEventNumber()):
        eventnum = trialacq.GetEvent(i)
        if eventnum.GetLabel()=="Foot Strike":
            if eventnum.GetContext() == "Left":
                patient["Gait"][TrialNum]["lfootstrikeFrame"].append(eventnum.GetFrame()-patient["Gait"][TrialNum]["startframe"])
            else:
                patient["Gait"][TrialNum]["rfootstrikeFrame"].append(eventnum.GetFrame()-patient["Gait"][TrialNum]["startframe"])
        if eventnum.GetLabel()=="Foot Off":
            if eventnum.GetContext() == "Left":
                patient["Gait"][TrialNum]["lfootoffFrame"].append(eventnum.GetFrame()-patient["Gait"][TrialNum]["startframe"])
            else:
                patient["Gait"][TrialNum]["rfootoffFrame"].append(eventnum.GetFrame()-patient["Gait"][TrialNum]["startframe"])

    for event in ["lfootstrikeFrame", "rfootstrikeFrame", "lfootoffFrame", "rfootoffFrame"]:
        patient["Gait"][TrialNum][event].sort()   
    
    if len(patient["Gait"][TrialNum]["lfootstrikeFrame"])>1:
        patient["Gait"][TrialNum]["Left"]["KneeAngle"] = cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["KneeAngle"])
        patient["Gait"][TrialNum]["Left"]["KneeForce"] = cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["KneeForce"])
        patient["Gait"][TrialNum]["Left"]["KneeMoment"] = cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["KneeMoment"])
        patient["Gait"][TrialNum]["Left"]["KneePower"] = cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["KneePower"])
        
        patient["Gait"][TrialNum]["Left"]["HipAngle"] = cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["HipAngle"])
        patient["Gait"][TrialNum]["Left"]["HipForce"] = cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["HipForce"])
        patient["Gait"][TrialNum]["Left"]["HipMoment"] = cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["HipMoment"])
        patient["Gait"][TrialNum]["Left"]["HipPower"] = cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["HipPower"])
        
        patient["Gait"][TrialNum]["Left"]["AnkleAngle"] = cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["AnkleAngle"])
        patient["Gait"][TrialNum]["Left"]["AnkleForce"] = cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["AnkleForce"])
        patient["Gait"][TrialNum]["Left"]["AnkleMoment"] = cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["AnkleMoment"])
        patient["Gait"][TrialNum]["Left"]["AnklePower"] = cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["AnklePower"])
    
        patient["Gait"][TrialNum]["Left"]["FootProgressAngle"] =cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["FootProgressAngle"])
        patient["Gait"][TrialNum]["Left"]["GRF"] =cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["GRF"])
        patient["Gait"][TrialNum]["Left"]["NGRF"] =cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["NGRF"])
        patient["Gait"][TrialNum]["Left"]["pelvisAngle"] =cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["pelvisAngle"])  
        patient["Gait"][TrialNum]["Left"]["COM"] =cutcycle(patient["Gait"][TrialNum]["lfootstrikeFrame"], patient["Gait"][TrialNum]["Left"]["COM"])
    #else:
        #print("no full left limb gait cycle")
    if len(patient["Gait"][TrialNum]["rfootstrikeFrame"])>1:
        patient["Gait"][TrialNum]["Right"]["KneeAngle"] = cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["KneeAngle"])
        patient["Gait"][TrialNum]["Right"]["KneeForce"] = cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["KneeForce"])
        patient["Gait"][TrialNum]["Right"]["KneeMoment"] = cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["KneeMoment"])
        patient["Gait"][TrialNum]["Right"]["KneePower"] = cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["KneePower"])
        
        patient["Gait"][TrialNum]["Right"]["HipAngle"] = cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["HipAngle"])
        patient["Gait"][TrialNum]["Right"]["HipForce"] = cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["HipForce"])
        patient["Gait"][TrialNum]["Right"]["HipMoment"] = cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["HipMoment"])
        patient["Gait"][TrialNum]["Right"]["HipPower"] = cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["HipPower"])
        
        patient["Gait"][TrialNum]["Right"]["AnkleAngle"] = cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["AnkleAngle"])
        patient["Gait"][TrialNum]["Right"]["AnkleForce"] = cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["AnkleForce"])
        patient["Gait"][TrialNum]["Right"]["AnkleMoment"] = cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["AnkleMoment"])
        patient["Gait"][TrialNum]["Right"]["AnklePower"] = cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["AnklePower"])
        
        patient["Gait"][TrialNum]["Right"]["FootProgressAngle"] =cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["FootProgressAngle"])
        patient["Gait"][TrialNum]["Right"]["GRF"] =cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["GRF"])
        patient["Gait"][TrialNum]["Right"]["NGRF"] =cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["NGRF"])
        patient["Gait"][TrialNum]["Right"]["pelvisAngle"] =cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["pelvisAngle"])
        patient["Gait"][TrialNum]["Right"]["COM"] =cutcycle(patient["Gait"][TrialNum]["rfootstrikeFrame"], patient["Gait"][TrialNum]["Right"]["COM"])
    # else:
    #     print("no full right limb gait cycle")
    
    for LR in ['Right','Left']:
        for key in patient["Gait"][TrialNum][LR].keys():
            arr = patient["Gait"][TrialNum][LR][key]
            patient["Gait"][TrialNum][LR][key] = {
                    'x':arr[:,0].tolist(),
                    'y':arr[:,1].tolist(),
                    'z':arr[:,2].tolist()
                    }
     
    #call other function"returnvalues" to extract rest of data
    #returnvalues(trialacq)
    global ID
    ID = trialacq.GetEvent(0).GetSubject()
    vID.set(ID)

repeated 6x

if there is a complete cycle:
- updates the label to say "Clean left (or right) gait cycle"
- updates variable cleanleft1 to say "Yes" instead of "No"


if there is not a complete cycle
- updates the label to say "Incomplete Left gait cycle"

In [ ]:
def cleanvalues1():
    if len(patient["Gait"]["Trial 1"]["lfootstrikeFrame"])>1:
        label_clean1.configure(text = "Clean Left Gait Cycle")
        global cleanleft1
        cleanleft1 = "Yes"
    else:
        label_clean1.configure(text = "Incomplete Left Gait Cycle")
    if len(patient["Gait"]["Trial 1"]["rfootstrikeFrame"])>1:
        label_clean1r.configure(text = "Clean Right Gait Cycle")
        global cleanright1
        cleanright1 = "Yes"
    else:
        label_clean1r.configure(text = "Incomplete Right Gait Cycle")
        

**new_index**\
The index values of the events are updated for the new cut data.\
Aslo, any negative events are deleted(events occuring before the first footstrike)

In [ ]:
def new_index(filename, TrialNum):
    if len(patient["Gait"][TrialNum]["lfootstrikeFrame"])>1:
        patient["Gait"][TrialNum]["lfootoffFrame"]=(np.array(patient["Gait"][TrialNum]["lfootoffFrame"])- 
                                                    (patient["Gait"][TrialNum]["lfootstrikeFrame"])[0]).tolist()
        patient["Gait"][TrialNum]["lfootstrikeFrame"]=(np.array(patient["Gait"][TrialNum]["lfootstrikeFrame"])-
                                                       (patient["Gait"][TrialNum]["lfootstrikeFrame"])[0]).tolist()
    for item in patient["Gait"][TrialNum]["lfootoffFrame"]:
        if item <0:
            patient["Gait"][TrialNum]["lfootoffFrame"].remove(item)
 
    if len(patient["Gait"][TrialNum]["rfootstrikeFrame"])>1:
        patient["Gait"][TrialNum]["rfootoffFrame"]=(np.array(patient["Gait"][TrialNum]["rfootoffFrame"])-
          (patient["Gait"][TrialNum]["rfootstrikeFrame"])[0]).tolist()
        patient["Gait"][TrialNum]["rfootstrikeFrame"]=(np.array(patient["Gait"][TrialNum]["rfootstrikeFrame"])-
          (patient["Gait"][TrialNum]["rfootstrikeFrame"])[0]).tolist()
    for item in patient["Gait"][TrialNum]["rfootoffFrame"]:
        if item <0:
            patient["Gait"][TrialNum]["rfootoffFrame"].remove(item)  

closes the window 

In [ ]:
def close_window():
    root.destroy()

# Widgets

set up the dictionary

In [ ]:
"""Initialise the Dict"""
patient = {}
patient["Gait"] = {}

Label at top of window with description of GUI

In [6]:
"""GUI Widgets"""
#FRAME 1: INFO
label_info1 = Label(FrameLabel, 
                    text = """Please select at least 3 Gait trials. 
                    Ensure a minimum of 3 complete gait cycles on each limb. """, 
                   bg = "ivory3", height = 2)
label_info1.grid()
label_info2 = Label(FrameLabel, 
                    text = "Input the patient data and select Accept to enter into the database.", 
                    bg = "ivory3", height = 2)
label_info2.grid()

This block of code is repeated 6 times for each of the trials. Could potentially be made into a class but then would be harder to call a function which then feeds back into the GUI (the labels)\

The button contains the text "Select Trial 1". When clicked, this calls the function "browse files 1" which brings up the file explorer. The file selected in an argument for the next function the button calls (extractintoDict) which breaks down the c3d file and extracts and sorts the data into a dictionary. It also detects the events(heelstrike, foot off) which are used to normalise the data (if complete clean cycle). If there is a complete clean cycle, the clean values function detects this and updates the label widget as to if the limb has a complete cycle or not.

The other widgets include the label for the filename and the labels for displaying if the limb has clean data or not.


In [7]:
#FRAMES 2-7: INDIVIDUAL TRIAL SELECTIONS
button_explore1 = Button(FrameTrial1, text = "Select Trial 1",  width = 12, bg = "ghost white",
                         command = lambda : [browseFiles1(), extractintoDict(filename1, "Trial 1"), 
                                             cleanvalues1(), new_index(filename1, "Trial 1")])  
label_file_explorer1 = Label(FrameTrial1,  text = "", width = 80, height = 3 ,  fg = "blue" , bg = "ghost white") 
button_explore1.grid(column = 1, row = 1, rowspan = 2) 
label_file_explorer1.grid(column = 2, row = 1, columnspan = 7, rowspan = 2) 
label_clean1 = Label(FrameTrial1, text="",  width = 25, bg = "ivory3")
label_clean1.grid(column = 10, row = 1)
label_clean1r = Label(FrameTrial1, text="",  width = 25, bg = "ivory3")
label_clean1r.grid(column = 10, row = 2)

The Frame containing th patient info inputs\
The select trial button automatically updates the Patient ID entry box, but can be overwritten\

A radiobutton option for inputting Dominant limb as "Left", "right" or "N/A" with the input saved as vDom\

A class exists for the widgets that are only applicable in some cases, which allow them to be disabled or normal.\
The widgets included in this class include a drop down menu (optionmenu) for the patient type. This is control, pre-op or post-op. It also includes a entry box for number of months post-op and a radiobutton for which limb is affected\
- if patient type is "Control", the entrybox and the radiobutton is disabled since neither limb is affected and they have had no operation
- if the patient type is "Pre-op" the radiobutton is normal to allow them to select the limb they will have an operation on, but the entry box is disabled
- if the patienttype is "Post-Op", both the radio button and the entry box are normal. Simply entry the number of months since the operation (if in years, calculate the number of months)


In [ ]:
#FRAME 8: INPuT PATIENT DATA
labelID = Label(FrameInfo,  text = "Patient ID: ", bg = "ivory3") 
labelID.grid(column = 1, row = 1)
vID = StringVar()
entryID = Entry(FrameInfo,  textvariable = vID,   fg = "purple4", width = 20) 
entryID.grid(column = 2, row = 1, columnspan = 3)
labelDom = Label(FrameInfo,  text = "Dominant Limb: ", bg = "ivory3") 
labelDom.grid(column = 1, row = 2)
vDom = StringVar()
vDom.set(0)
radioleftdom = Radiobutton(FrameInfo, text="Left",variable= vDom, value="Left")
radioleftdom.grid(column = 2, row = 2)
radiorightdom = Radiobutton(FrameInfo, text="Right",variable =vDom, value="Right")
radiorightdom.grid(column = 3, row = 2)
radiorightdom = Radiobutton(FrameInfo, text="N/A",variable =vDom, value="N/A")
radiorightdom.grid(column = 4, row = 2)

class Pattype:
    def disable(self, master):
        if vtype.get() == "Post-op":
            self.month_entry.configure(state = "normal")
            self.date_label.configure(state = "normal")
            self.limb_label.configure(state = "normal")
            self.radioleft.configure(state = "normal")
            self.radioright.configure(state = "normal")
            self.radioboth.configure(state = "normal")
        if vtype.get() == "Control":
            self.month_entry.configure(state = "disabled")
            self.date_label.configure(state = "disabled")
            self.limb_label.configure(state = "disabled")
            self.radioleft.configure(state = "disabled")
            self.radioright.configure(state = "disabled")
            self.radioboth.configure(state = "disabled")
        if vtype.get() == "Pre-op":
            self.month_entry.configure(state = "disabled")
            self.date_label.configure(state = "disabled")    
            self.limb_label.configure(state = "normal")
            self.radioleft.configure(state = "normal")
            self.radioright.configure(state = "normal")
            self.radioboth.configure(state = "normal")
            
    def __init__(self, master):
        self.label_type = Label(FrameInfo, text = "Patient Type: ", bg = "ivory3")
        self.label_type.grid(column = 1, row = 3)
        global vtype
        vtype = StringVar()
        self.optiontype = OptionMenu(FrameInfo, vtype, "Pre-op", "Post-op", "Control", 
                                     command = self.disable)
        self.optiontype.config(width = 20)
        self.optiontype.grid(column = 2, row = 3, columnspan  =3)
        
        self.date_label = Label(FrameInfo, text = "Number of months Post - Op: ", bg = "ivory3")
        self.date_label.grid(column = 1, row  = 4)
        global vmonth
        vmonth = IntVar()
        self.month_entry = Entry(FrameInfo, textvariable = vmonth, width = 20, fg = "purple4")
        self.month_entry.grid(column = 2, row = 4, columnspan  = 3)
        
        self.limb_label = Label(FrameInfo, text = "Affected Limb: ", bg = "ivory3")
        self.limb_label.grid(column = 1, row  = 5)
        global vlimb
        vlimb = StringVar()
        vlimb.set(0)
        self.radioleft = Radiobutton(FrameInfo, text="Left",variable= vlimb, value="Left")
        self.radioleft.grid(column = 2, row = 5)
        self.radioright = Radiobutton(FrameInfo, text="Right",variable=vlimb, value="Right")
        self.radioright.grid(column = 3, row = 5)
        self.radioboth = Radiobutton(FrameInfo, text="Both",variable=vlimb, value="Both")
        self.radioboth.grid(column = 4, row = 5)     
Pattype(FrameInfo)

inputs patient data into Patient_Data table. If less than 6 files, will input filenames as Null or None values

In [ ]:
def enter_patient_data():
    c.execute("""INSERT INTO Patient_Data
              (ID, Dominant_Limb, Patient_Type, Month, Affected_Limb,
              Filename_Gait_Trial_1, Filename_Gait_Trial_2, Filename_Gait_Trial_3,
              Filename_Gait_Trial_4, Filename_Gait_Trial_5, Filename_Gait_Trial_6)
              VALUES (?,?,?,?,?,?,?,?,?,?,?)""",
              (vID.get(), vDom.get(), vtype.get(), vmonth.get(), vlimb.get(),
               filename1, filename2, filename3, filename4, filename5, filename6))

input data into Gait_table

**NOTE**: The list of the full datasets are turned into strings and the open and close bracket are removed. To turn back into a list just use the split fucntion with the deliminator = ","

In [ ]:
def enter_Gait_Table (numTrials):
    for TrialNum in numTrials:
        if TrialNum=="Trial 1":
            filename = filename1
            cleanleft = cleanleft1
            cleanright = cleanright1
        if TrialNum=="Trial 2":
            filename = filename2
            cleanleft = cleanleft2
            cleanright = cleanright2
        if TrialNum=="Trial 3":
            filename = filename3
            cleanleft = cleanleft3
            cleanright = cleanright3
        if TrialNum=="Trial 4":
            filename = filename4
            cleanleft = cleanleft4
            cleanright = cleanright4
        if TrialNum=="Trial 5":
            filename = filename5
            cleanleft = cleanleft5
            cleanright = cleanright5
        if TrialNum=="Trial 6":
            filename = filename6
            cleanleft = cleanleft6
            cleanright = cleanright6
         
        c.execute("""INSERT INTO Gait_Table 
                  (Filename, Complete_left_Cycle, Complete_Right_Cycle, 
                  Left_Footstrike_Index, Left_FootOff_Index, 
                  Right_Footstrike_Index, Right_FootOff_Index) 
                  VALUES (?,?,?,?,?,?,?)""", (filename, cleanleft, cleanright,
                  (str(patient["Gait"][TrialNum]["lfootstrikeFrame"])),
                  (str(patient["Gait"][TrialNum]["lfootoffFrame"])),
                  (str(patient["Gait"][TrialNum]["rfootstrikeFrame"])),
                  (str(patient["Gait"][TrialNum]["rfootoffFrame"]))))
        for LR in ["Left", "Right"]:
            for axis in ["x", "y", "z"]:
                c.execute(f"""UPDATE Gait_Table
                          SET 
                          {LR}_Knee_Angle_{axis} = ?,
                          {LR}_Knee_Moment_{axis} = ?,
                          {LR}_Knee_Force_{axis} = ?,
                          {LR}_Knee_Power_{axis} = ?,
                          {LR}_Hip_Angle_{axis} = ?,
                          {LR}_Hip_Moment_{axis} = ?,
                          {LR}_Hip_Force_{axis} = ?,
                          {LR}_Hip_Power_{axis} = ?,
                          {LR}_Ankle_Angle_{axis} = ?,
                          {LR}_Ankle_Moment_{axis} = ?,
                          {LR}_Ankle_Force_{axis} = ?,
                          {LR}_Ankle_Power_{axis} =  ?,
                          {LR}_Foot_Progression_Angle_{axis} = ?,
                          {LR}_GRF_{axis} =  ?,
                          {LR}_NGRF_{axis} =  ?,
                          {LR}_pelvis_Angle_{axis} =  ?,
                          {LR}_COM_{axis} =  ?
                          WHERE Filename = ?
                          """,(
                          (str(patient["Gait"][TrialNum][LR]["KneeAngle"][axis])[1:-1]),
                          (str(patient["Gait"][TrialNum][LR]["KneeMoment"][axis])[1:-1]),
                          (str(patient["Gait"][TrialNum][LR]["KneeForce"][axis])[1:-1]),
                          (str(patient["Gait"][TrialNum][LR]["KneePower"][axis])[1:-1]),
                          (str(patient["Gait"][TrialNum][LR]["HipAngle"][axis])[1:-1]),
                          (str(patient["Gait"][TrialNum][LR]["HipMoment"][axis])[1:-1]),
                          (str(patient["Gait"][TrialNum][LR]["HipForce"][axis])[1:-1]),
                          (str(patient["Gait"][TrialNum][LR]["HipPower"][axis])[1:-1]),
                          (str(patient["Gait"][TrialNum][LR]["AnkleAngle"][axis])[1:-1]),
                          (str(patient["Gait"][TrialNum][LR]["AnkleMoment"][axis])[1:-1]),
                          (str(patient["Gait"][TrialNum][LR]["AnkleForce"][axis])[1:-1]),
                          (str(patient["Gait"][TrialNum][LR]["AnklePower"][axis])[1:-1]),
                          (str(patient["Gait"][TrialNum][LR]["FootProgressAngle"][axis])[1:-1]),
                          (str(patient["Gait"][TrialNum][LR]["GRF"][axis])[1:-1]),
                          (str(patient["Gait"][TrialNum][LR]["NGRF"][axis])[1:-1]),
                          (str(patient["Gait"][TrialNum][LR]["pelvisAngle"][axis])[1:-1]),
                          (str(patient["Gait"][TrialNum][LR]["COM"][axis])[1:-1]),
                          filename
                          ))


Final Frame with two buttons: Accept and Cancel. 
- "Accept" will add the data into the database and close the window
- "Cancel" will just close the window without adding into the database

In [ ]:
buttonaccept = Button(FrameClose, text = "Accept", 
                      command  = lambda: [enter_patient_data(), enter_Gait_Table(numTrialslist), close_window()])
buttonaccept.grid(column = 1, row = 1, padx = 10, pady = 20)
buttoncancel = Button(FrameClose, text = "Cancel", command = lambda: close_window())
buttoncancel.grid(column=2, row  = 1, padx = 10)

close the GUI by running the mainloop

In [ ]:
root.mainloop()

writes the dictionary into a unique filename which can be used as backup?

In [ ]:
"""Write Dict to JSON"""
name = ID + pattype + str(month) + ".json"
with open(name, 'w') as f:
      json.dump(patient, f, indent = 5)


Prints everythig in the both tables to make sure it has been inputted into the SQLite database

In [ ]:
def selectall():
    c.execute("""SELECT * FROM Gait_Table""")
    rows = c.fetchall()
    for row in rows:
        print(row)

def selectall2():
    c.execute("""SELECT * FROM Patient_Data""")
    lines = c.fetchall()
    for line in lines:
        print(line)
        
        
def selectall3():
    c.execute("""SELECT Filename, 
              Complete_Left_Cycle, Complete_Right_Cycle, 
              Left_Footstrike_Index, Left_FootOff_Index,
              Right_Footstrike_Index, Right_FootOff_Index 
              FROM Gait_Table""")
    lines = c.fetchall()
    for line in lines:
        print(line)
        #
#selectall()  
selectall2()
selectall3()


Close SQL

In [ ]:
"""Close SQLite Connection"""
conn.commit()
conn.close()   